In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from xgboost import XGBClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import cohen_kappa_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.experimental import enable_halving_search_cv
#from sklearn.model_selection import HalvingRandomSearchCV


In [2]:
test_run = True
#test_run = False

In [3]:
df = pd.read_csv("Temp/df_proc_train_set.csv")

In [4]:
df['sol_category'] = pd.to_numeric(df['sol_category'], downcast='integer')
df

,Id,smiles,sol_category,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,EOS12286,Cc1nc(N2CCN(C(=O)Nc3ccc(F)cc3F)CC2)cc(-n2ccnc2)n1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,EOS85869,CCN(CC)[C@H]1CCN(C(=O)Cc2nc(C(C)C)c(C)s2)C1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,EOS85435,CNC(=O)CNC(=O)c1c(-n2cccc2)sc(C)c1C,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,EOS102302,CC(C)(C)c1ccc(CSc2cnn(C(C)(C)C)c(=O)c2Cl)cc1,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EOS64213,CC[C@H](NC(=O)c1ccnc(-n2ccnc2)c1)c1ccccc1OC,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70706,EOS37839,O=C(NCCCc1nc(=O)[nH][nH]1)[C@H]1CCC(F)(F)C1,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
70707,EOS2088,Cc1ccc(C(=O)NC2CCCC2)cc1S(=O)(=O)N1CCOCC1,2,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70708,EOS10587,COCCN1CCC(CN(C)S(=O)(=O)c2cccc(C(F)(F)F)c2)C1,2,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70709,EOS40533,O=C(Nc1ccc(F)cc1)NC1CCN(C(=O)Cc2cnn(-c3ccccc3)...,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [5]:
if test_run:
    #df0 = pd.DataFrame()
    #index = 0
    #for index in tqdm(df.index):
    #    if index % 100 == 0:
    #        #row_i = df.loc[index, :]
    #        df0 = pd.concat([df0, df.loc[index, :]], axis=1) #, row_i])
    #df0 = df0.T
    #df0['sol_category'].value_counts()
    #df = df0
    df = df.head(5000)

In [6]:
df

,Id,smiles,sol_category,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,EOS12286,Cc1nc(N2CCN(C(=O)Nc3ccc(F)cc3F)CC2)cc(-n2ccnc2)n1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,EOS85869,CCN(CC)[C@H]1CCN(C(=O)Cc2nc(C(C)C)c(C)s2)C1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,EOS85435,CNC(=O)CNC(=O)c1c(-n2cccc2)sc(C)c1C,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,EOS102302,CC(C)(C)c1ccc(CSc2cnn(C(C)(C)C)c(=O)c2Cl)cc1,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EOS64213,CC[C@H](NC(=O)c1ccnc(-n2ccnc2)c1)c1ccccc1OC,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,EOS86615,O=C(C1CC=CC1)N1CCC[C@H](c2cc(C(F)(F)F)[nH]n2)C1,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,EOS90873,CCOc1ccc(-c2csc(CCN3CCOC3=O)n2)cc1,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4997,EOS20619,NC(=O)c1ccccc1NC(=O)c1cccc(I)c1,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4998,EOS63886,Cc1ccccc1CN(C)S(=O)(=O)c1cc(C(N)=O)sc1C,2,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [7]:
#y = pd.read_excel(xl, 'y', header = None)

In [8]:
# hypothesis fuction
# takes input variables and theta, return calculated output variable

In [9]:
print(len(df))
#print(len(y))
X = df.copy(deep=True)
X.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X

5000


,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,ecfp_7,ecfp_8,ecfp_9,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4997,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4998,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Only exfp

In [10]:
X_ecfp = df.copy(deep=True)
drop_list_desc = [column for column in X_ecfp.columns if (column.startswith('example_') or column.startswith('rdkit_desc_'))]

# Drop features 
X_ecfp.drop(drop_list_desc, axis=1, inplace=True)
X_ecfp.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X_ecfp

,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,ecfp_7,ecfp_8,ecfp_9,...,ecfp_4086,ecfp_4087,ecfp_4088,ecfp_4089,ecfp_4090,ecfp_4091,ecfp_4092,ecfp_4093,ecfp_4094,ecfp_4095
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Only descriptors

In [11]:
X_desc = df.copy(deep=True)
drop_list_MFP = [column for column in X_desc.columns if column.startswith('ecfp_')]

# Drop features 
X_desc.drop(drop_list_MFP, axis=1, inplace=True)
X_desc.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X_desc

,example_SlogP,example_SMR,example_TPSA,example_NumLipinskiHBA,example_NumLipinskiHBD,example_NumRotatableBonds,example_NumHBA,example_NumAmideBonds,example_NumHeteroAtoms,example_NumAtoms,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,2.60302,103.0537,79.18,8,1,3,6,2,10,48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,3.06012,92.5500,36.44,4,0,6,4,1,5,51,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.63144,79.5199,63.13,5,2,4,4,2,6,37,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,5.24150,103.2060,34.89,3,0,3,4,0,5,49,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.15700,94.8512,69.04,6,1,6,5,1,6,45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3.10070,74.0087,48.99,4,1,2,2,1,7,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,3.20350,85.3440,51.66,5,0,6,5,1,6,40,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4997,2.64240,82.0721,72.19,4,3,3,2,2,5,30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4998,2.28454,87.4057,80.47,5,2,5,4,1,7,40,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


#### only example descriptors

In [12]:
X_example = X_desc.copy(deep=True)
drop_list_ex = [column for column in X_example.columns if column.startswith('rdkit_desc_')]

# Drop features 
X_example.drop(drop_list_ex, axis=1, inplace=True)
#X_example.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X_example

,example_SlogP,example_SMR,example_TPSA,example_NumLipinskiHBA,example_NumLipinskiHBD,example_NumRotatableBonds,example_NumHBA,example_NumAmideBonds,example_NumHeteroAtoms,example_NumAtoms,...,example_NumSaturatedRings,example_NumAliphaticRings,example_NumAromaticHeterocycles,example_NumSaturatedHeterocycles,example_NumAliphaticHeterocycles,example_NumAromaticCarbocycles,example_NumSaturatedCarbocycles,example_NumAliphaticCarbocycles,example_FractionCSP3,example_HallKierAlpha
0,2.60302,103.0537,79.18,8,1,3,6,2,10,48,...,1,1,2,1,1,1,0,0,0.263158,-3.56
1,3.06012,92.5500,36.44,4,0,6,4,1,5,51,...,1,1,1,1,1,0,0,0,0.764706,-0.94
2,1.63144,79.5199,63.13,5,2,4,4,2,6,37,...,0,0,2,0,0,0,0,0,0.285714,-2.08
3,5.24150,103.2060,34.89,3,0,3,4,0,5,49,...,0,0,1,0,0,1,0,0,0.473684,-1.26
4,3.15700,94.8512,69.04,6,1,6,5,1,6,45,...,0,0,2,0,0,1,0,0,0.210526,-3.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3.10070,74.0087,48.99,4,1,2,2,1,7,40,...,1,2,1,1,1,0,0,1,0.600000,-1.79
4996,3.20350,85.3440,51.66,5,0,6,5,1,6,40,...,1,1,1,1,1,1,0,0,0.375000,-2.08
4997,2.64240,82.0721,72.19,4,3,3,2,2,5,30,...,0,0,0,0,0,2,0,0,0.000000,-1.89
4998,2.28454,87.4057,80.47,5,2,5,4,1,7,40,...,0,0,1,0,0,1,0,0,0.266667,-1.70


#### only rdkit descriptors

In [13]:
X_rdd = X_desc.copy(deep=True)
drop_list_rd = [column for column in X_rdd.columns if column.startswith('example_')]

# Drop features 
X_rdd.drop(drop_list_rd, axis=1, inplace=True)
#X_example.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X_rdd

,rdkit_desc_MaxEStateIndex,rdkit_desc_MinEStateIndex,rdkit_desc_MinAbsEStateIndex,rdkit_desc_qed,rdkit_desc_MaxPartialCharge,rdkit_desc_MinPartialCharge,rdkit_desc_MaxAbsPartialCharge,rdkit_desc_FpDensityMorgan1,rdkit_desc_FpDensityMorgan2,rdkit_desc_FpDensityMorgan3,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,13.751480,-0.805113,0.046085,0.732517,0.321549,-0.352889,0.352889,1.068966,1.827586,2.517241,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,12.524029,0.234919,0.234919,0.807124,0.229004,-0.340705,0.340705,1.409091,2.090909,2.636364,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,12.317465,-0.223009,0.019061,0.899102,0.254519,-0.357642,0.357642,1.250000,1.900000,2.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,12.357761,-0.382235,0.149177,0.698158,0.286663,-0.266023,0.286663,1.083333,1.666667,2.166667,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12.704152,-0.151369,0.131009,0.750601,0.251455,-0.496395,0.496395,1.280000,2.120000,2.840000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,12.634965,-4.411766,0.003398,0.853008,0.432287,-0.341770,0.432287,1.318182,2.045455,2.681818,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,11.382228,-0.219174,0.219174,0.820516,0.409449,-0.493889,0.493889,1.363636,2.181818,2.863636,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4997,12.074562,-0.570794,0.271096,0.820274,0.255216,-0.365532,0.365532,1.052632,1.789474,2.473684,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4998,12.685055,-3.661322,0.141096,0.908009,0.258321,-0.364900,0.364900,1.272727,1.909091,2.454545,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [14]:
y = df.iloc[:, 2]

In [15]:
#y

In [16]:
y.unique()

array([0, 1, 2], dtype=int8)

## StandardScaler

In [17]:
#scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(X)
#X = scaler.fit_transform(X)

## split

In [18]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=1,  shuffle=True, stratify=y)

In [19]:
# find out where NaN is, remove in processing

## multi-class logistic regression

In [20]:
#clf = LogisticRegression(random_state=0, max_iter=1000, verbose=1).fit(X_train, y_train)

In [21]:
#clf.predict(X[:2, :])

In [22]:
#clf.predict_proba(X[:2, :])

In [23]:
#y_prediction = clf.predict(X_test)

In [24]:
#df_y = pd.DataFrame(y_prediction)

In [25]:
# Stricter feature selection
### 1. just use ecfp/rdkit+example
### 2. Hyperparatuning (StratifiedKfold+RandomizedSearchCV on X_train and y_train) (max_iter, solver, penalty,C)

# for dataset over all_dataset
#   hyperparameter tuning()

In [26]:
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import cohen_kappa_score
from numpy import linspace

In [27]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

import warnings
warnings.filterwarnings("ignore", category=UserWarning)


In [28]:
all_dataset = list()
all_dataset.append(X)
all_dataset.append(X_ecfp)
all_dataset.append(X_desc)
#all_dataset.append(X_example)
#all_dataset.append(X_rdd)

In [29]:
# setup parameters for xgboost
param = {}
param['booster'] = 'gbtree'
param['objective'] = 'binary:logistic'
param["eval_metric"] = "error"
param['eta'] = 0.3
param['gamma'] = 0
param['max_depth'] = 6
param['min_child_weight']=1
param['max_delta_step'] = 0
param['subsample']= 1
param['colsample_bytree']=1
param['silent'] = 1
param['seed'] = 0
param['base_score'] = 0.5



In [30]:
#qwck_scorer = make_scorer(cohen_kappa_score, weights='quadratic')

#predictions = list()
qwck_scorer = make_scorer(cohen_kappa_score, weights='quadratic')

print('------------------------------------------------------')

for dataset in all_dataset:
    # apply StandardScaler
    scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(dataset)
    dataset = scaler.fit_transform(dataset)

    # split dataset
    StratifiedKFold(n_splits=5)
    X_train, X_test, y_train, y_test = train_test_split(dataset, y, random_state=1,  shuffle=True, stratify=y)#, test_size=0.2, train_size=0.8)

    # method
    model = XGBClassifier()#**param)
    randm_src = RandomizedSearchCV(estimator=model, param_distributions = param, cv = 5, n_iter = 10, n_jobs=5, scoring=qwck_scorer, verbose=2)
    randm_src.fit(X_train, y_train)

    # output
    print(f'results of data set {dataset}')
    print('------------------------------------------------------')
    print(" Results from Random Search: " )
    print("\n The best estimator across ALL searched params:\n", randm_src.best_estimator_)
    print("\n The best score across ALL searched params:\n", randm_src.best_score_)
    print("\n The best parameters across ALL searched params:\n", randm_src.best_params_)
    print('------------------------------------------------------------------------------------------------------------------------------------------------------------------')


------------------------------------------------------


TypeError: Parameter grid for parameter 'eta' is not iterable or a distribution (value=0.3)

## Score

In [ ]:
#len(testPredictions)
#testPredictions

In [ ]:
#y_test

In [ ]:
set(predictions[0])

{1}

In [ ]:
'''i=1

for preds in predictions:
    # defining name
    if i == 1:
        dataset_name = 'X'
    elif i == 2:
        dataset_name = 'X_exfp'
    elif i == 3:
        dataset_name = 'X_desc'
    
    i += 1

    #quad kappa score
    sk_quad_kappa = cohen_kappa_score(y_test, testPredictions, weights='quadratic')
    
    # printing results
    print(f'quadratic kappa score of data set {dataset_name}: {sk_quad_kappa}')'''

quadratic kappa score of data set X: 0.0
quadratic kappa score of data set X_exfp: 0.0
quadratic kappa score of data set X_desc: 0.0


In [ ]:
#sk_cohen_kappa = cohen_kappa_score(y_prediction, y_test, weights='quadratic')
#print(f'quadratic cohen kappa: {sk_cohen_kappa}')

In [ ]:
#from sklearn.metrics import confusion_matrix

In [ ]:
#confusion_matrix(y_test, y_prediction)

In [ ]:
#df_y[0].value_counts()

In [ ]:
#len(y_test)